In [1]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv

load_dotenv()
league_id = 91606
url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/2020/segments/0/leagues/{}".format(league_id)
cookies = {
        "swid": os.getenv('swid'),
        "espn_s2": os.getenv('espn_s2')
    }


In [2]:
games_r = requests.get(url,
params={"view": "mMatchup"},
    cookies=cookies)
schedule = games_r.json()["schedule"]


In [3]:
games_df = pd.DataFrame([[game["matchupPeriodId"], 
game["home"]["teamId"], game["home"]["totalPoints"],
game["away"]["teamId"], game["away"]["totalPoints"],] for game in schedule],
    columns=['Week', 'Team1', 'Score1', 'Team2', 'Score2'])
games_df.head()

,Week,Team1,Score1,Team2,Score2
0,1,1,139.0,13,109.6
1,1,12,146.8,9,122.8
2,1,16,129.4,3,129.5
3,1,10,142.3,15,167.0
4,1,7,145.3,17,110.7


In [4]:
game_margin_df = games_df.assign(Margin1 = games_df["Score1"] - games_df["Score2"],
    Margin2 = games_df["Score2"] - games_df["Score1"])
game_margin_df.head()

,Week,Team1,Score1,Team2,Score2,Margin1,Margin2
0,1,1,139.0,13,109.6,29.4,-29.4
1,1,12,146.8,9,122.8,24.0,-24.0
2,1,16,129.4,3,129.5,-0.1,0.1
3,1,10,142.3,15,167.0,-24.7,24.7
4,1,7,145.3,17,110.7,34.6,-34.6


In [5]:
g_flattened = (game_margin_df[['Week', 'Team1', 'Margin1', 'Score1']]
    .rename(columns={'Team1': 'Team', 'Score1': 'Score', 'Margin1': 'Margin'})
    .append(game_margin_df[['Week', 'Team2', 'Margin2', 'Score2']]
    .rename(columns={'Team2': 'Team', 'Score2': 'Score', 'Margin2': 'Margin'})
    ))

g_flattened.to_csv("games.csv")